# Function calling

Introduction

In an API call, you can describe functions and have the model intelligently choose to output a JSON object containing arguments to call one or many functions. The Chat Completions API does not call the function; instead, the model generates JSON that you can use to call the function in your code.


In [1]:
import os
from openai import AzureOpenAI
import json
from dotenv import load_dotenv

load_dotenv("azure.env")

# gets API Key from environment variable OPENAI_API_KEY
client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
  api_version=os.getenv("API_VERSION")
)

## Define functions that will take actions like retrieve information from internet

In [2]:
# Example function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": unit})
    elif "san francisco" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": unit})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": unit})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})


## Let's ask a question

In [4]:
messages = [{"role": "user", "content": "What's the weather like in San Francisco, Tokyo, and Paris?"}]
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },
    }
]
response = client.chat.completions.create(
    model=os.getenv("MODEL_VERSION"),
    messages=messages,
    tools=tools,
    tool_choice="auto",  # auto is default, but we'll be explicit
)
response_message = response.choices[0].message
tool_calls = response_message.tool_calls

# Step 3: call the function
# Note: the JSON response may not always be valid; be sure to handle errors
available_functions = {
    "get_current_weather": get_current_weather,
}  # only one function in this example, but you can have multiple
messages.append(response_message)  # extend conversation with assistant's reply

# Step 4: send the info for each function call and function response to the model
for tool_call in tool_calls:
    function_name = tool_call.function.name
    function_to_call = available_functions[function_name]
    function_args = json.loads(tool_call.function.arguments)
    function_response = function_to_call(
        location=function_args.get("location"),
        unit=function_args.get("unit"),
    )
    messages.append(
        {
            "tool_call_id": tool_call.id,
            "role": "tool",
            "name": function_name,
            "content": function_response,
        }
    )  # extend conversation with function response
    print(messages)
second_response = client.chat.completions.create(
    model=os.getenv("MODEL_VERSION"),
    messages=messages,
)  
# get a new response from the model where it can see the function response
print("Answer:")
print(second_response.choices[0].message.content)  # print the assistant's response

[{'role': 'user', 'content': "What's the weather like in San Francisco, Tokyo, and Paris?"}, ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_jivIhmfbnZnY6Gbyh2ky6Lf8', function=Function(arguments='{"location":"San Francisco, CA","unit":"fahrenheit"}', name='get_current_weather'), type='function')]), {'tool_call_id': 'call_jivIhmfbnZnY6Gbyh2ky6Lf8', 'role': 'tool', 'name': 'get_current_weather', 'content': '{"location": "San Francisco", "temperature": "72", "unit": "fahrenheit"}'}]
Answer:
In San Francisco, it is currently 72 degrees Fahrenheit. I will now check the weather in Tokyo and Paris.
